In [1]:
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.{min, max, mean, stddev, typedLit, when}
import org.apache.spark.sql.types.{IntegerType, DoubleType}

println("all good")

all good


data is in the format:

{
"conversionSymbol": "",
"volumeto": 6685400.72, 
"high": 9165.89, 
"low": 9144.6, 
"time": 1595030400, 
"volumefrom": 730.39, 
"close": 9151.68, 
"open": 9156.79, 
"conversionType": "direct"
}


In [2]:
case class TradingInfo (time: Int,
                        high: Double,
                        low: Double,
                        open: Double,
                        volumefrom: Double,
                        volumeto: Double, 
                        close: Double,
                        conversionType: String,
                        conversionSymbol: String                        
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1).toInt, s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toDouble, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1), s(8).split(": ")(1))
}

val filename = "../Datasets/bitcoin_json.txt"
//val johnFileName = "//home/john/ID2221-Labs/PreProcessing/"
val tradingInformation = sc.textFile(filename).
            flatMap(l => l.split("},")).
            map(x => x.replace("[", "")).
            map(x => x.replace("{", "")).
            map(x => x.replace("\"", "")).
            map(createTradingInfo)
            .cache()

tradingInformation.take(3).foreach(println)

println("All good")

TradingInfo(1595466000,9540.53,9515.47,9524.38,1555.82,1.483120278E7,9538.04,\direct\,\\)
TradingInfo(1595469600,9545.07,9512.26,9538.04,1358.04,1.293527787E7,9514.73,\direct\,\\)
TradingInfo(1595473200,9514.84,9486.07,9514.73,1372.42,1.303854406E7,9508.16,\direct\,\\)
All good


defined class TradingInfo
filename = ../Datasets/bitcoin_json.txt
tradingInformation = MapPartitionsRDD[6] at map at <console>:40


createTradingInfo: (data: String)TradingInfo


MapPartitionsRDD[6] at map at <console>:40

In [3]:
// Data Action

val df = spark.createDataFrame(tradingInformation).sort("time")
val df1 = df.withColumn("midPrice",($"high" + $"low")/2)

// df1.show(5)

val ratio: Double = 0.9
val training_size = df1.count*ratio
val testing_size = df1.count - 10

val training_position = df1.take(training_size.toInt).last.getInt(0)
val testing_position = df1.take(testing_size.toInt).last.getInt(0)

// val df2 = df1.withColumn("dataset", when(col("time") <= training_position,"training")
//       .when(col("time") > training_position and col("time") <= testing_position,"validation")
//       .otherwise("testing"))
// df2.show(5)


val training_set = df1.filter($"time" <= training_position).cache()
val validation_set = df1.filter($"time" > training_position).filter($"time" <= testing_position).cache()
val testing_set = df1.filter($"time" > testing_position).cache()


// training_set.show(5)
// validation_set.show(5)
// testing_set.show(5)


println(testing_set.count)

println("All good")

10
All good


df = [time: int, high: double ... 7 more fields]
df1 = [time: int, high: double ... 8 more fields]
ratio = 0.9
training_size = 53851.5
testing_size = 59825
training_position = 1582441200
testing_position = 1602626400
training_set = [time: int, high: double ... 8 more fields]
validation_set = [time: int, high: double ... 8 more fields]
testing_set = [time: int, high: double ... 8 more fields]


[time: int, high: double ... 8 more fields]

In [19]:
// training_set
// validation_set
// testing_set
val data_type = "cleaned_data/training_set"
val dataset = training_set

val statistics = dataset.agg(mean("midPrice"), stddev("midPrice"), min("midPrice"), max("midPrice"),
                                  mean("volumefrom"), stddev("volumefrom"), min("volumefrom"), max("volumefrom"),
                                  mean("volumeto"), stddev("volumeto"), min("volumeto"), max("volumeto")).head()

val midPrice_mean = statistics.getDouble(0)
val midPrice_std = statistics.getDouble(1)
val midPrice_min = statistics.getDouble(2)
val midPrice_max = statistics.getDouble(3)
val midPrice_range = midPrice_max - midPrice_min

// // VolumeFrom
val volumefrom_mean = statistics.getDouble(4)
val volumefrom_std = statistics.getDouble(5)
val volumefrom_min = statistics.getDouble(6)
val volumefrom_max = statistics.getDouble(7)
val volumefrom_range = volumefrom_max - volumefrom_min

// // VolumeTo
val volumeto_mean = statistics.getDouble(8)
val volumeto_std = statistics.getDouble(9)
val volumeto_min = statistics.getDouble(10)
val volumeto_max = statistics.getDouble(11)
val volumeto_range = volumeto_max - volumeto_min

// // // from -1 to 1 
// // //val new_dataset = dataset.withColumn("midPriceNorm",($"midPrice" - midPrice_mean)/midPrice_std)
// // // from 0 to 1 
val new_dataset = dataset.withColumn("midPriceNorm",(($"midPrice" - midPrice_min)/midPrice_range).cast(DoubleType))
               .withColumn("volumefromNorm",(($"volumefrom" - volumefrom_min)/volumefrom_range).cast(DoubleType))
               .withColumn("volumetoNorm",(($"volumeto" - volumeto_min)/volumeto_range).cast(DoubleType))
               .cache()

new_dataset.show(5)
new_dataset.printSchema()

println("All good")

+----------+-------+------+-------+----------+--------+-------+--------------+----------------+-----------------+--------------------+--------------------+--------------------+
|      time|   high|   low|   open|volumefrom|volumeto|  close|conversionType|conversionSymbol|         midPrice|        midPriceNorm|      volumefromNorm|        volumetoNorm|
+----------+-------+------+-------+----------+--------+-------+--------------+----------------+-----------------+--------------------+--------------------+--------------------+
|1388581200|  967.0|718.46|  812.9|     1.012|  889.99|  967.0|      \direct\|              \\|           842.73| 0.03430077268445006|1.203243540332955...|3.860005357360329E-7|
|1388584800| 759.11|750.13| 750.13|     1.661| 1282.36| 759.11|      \direct\|              \\|           754.62|0.029799236920461117|1.974888854242133...|5.561766390706178E-7|
|1388588400|1169.16|912.66|1169.16|    0.5916|  575.07|1000.26|      \direct\|              \\|          1040.91| 0

data_type = cleaned_data/training_set
dataset = [time: int, high: double ... 8 more fields]
statistics = [3506.103169300478,3871.254468896739,171.35000000000002,19744.67,3401.369493895933,55179.039868938766,0.0,8410599.9,1.172089303648103E7,2.980054803564982E7,0.0,2.30567037505E9]
midPrice_mean = 3506.103169300478
midPrice_std = 3871.254468896739
midPrice_min = 171.35000000000002
midPrice_max = 19744.67
midPrice_range = 19573.32
volumefrom_mean = 3401.369493895933
volumefrom_std = 55179.039868938766
volumefrom_min = 0.0
volumefrom_max = 8410599.9
volumefrom_range = 8410599.9
volumeto_mean = 1.172089303648103E7


v...


1.172089303648103E7

In [20]:
// Saving data as Text

// don't forget to delete the previous folder 
// RDD does not allow us to overwrite: https://community.cloudera.com/t5/Support-Questions/Apache-SPARK-Overwrite-data-file/m-p/105253
val pre_processed_dataset = new_dataset.withColumn("empty", typedLit(Seq(1)))
pre_processed_dataset.rdd.repartition(1).saveAsTextFile(data_type)

println("All good")

All good


pre_processed_dataset = [time: int, high: double ... 12 more fields]


[time: int, high: double ... 12 more fields]